# Machine Translation

In [1]:
!nvidia-smi
!lscpu

Wed Jul  7 11:55:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install hazm==0.7.0
!pip install seqeval==1.2.2
!pip install sentencepiece==0.1.96
!pip install transformers==4.7.0
!pip install clean-text[gpl]==0.4.0
!pip install sacrebleu==1.5.1

     |████████████████████████████████| 317kB 33.3MB/s 
     |████████████████████████████████| 235kB 44.0MB/s 
     |████████████████████████████████| 1.4MB 46.0MB/s 
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154158 sha256=1434f8aeeff7d117618d4b6a08e61d584e0b3bde8e3878e88cd95d05acb71ef6
  Stored in directory: /root/.cache/pip/wheels/66/15/54/4510dce8bb958b1cdd2c47425cbd1e1eecc0480ac9bb1fb9ab
  Created wheel for nltk: filename=nltk-3.3-cp37-none-any.whl size=1394486 sha256=bce5a8d01471d61c000aef7ff4269aad667b5c5dba8cc31425b1f321e1b8fe43
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
Successfully built libwapiti nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 51kB 7.4MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16184 sha256=c660b6144382a4

In [3]:
!pip install PyDrive
import os
import IPython.display as ipd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
# Import required packages
import os
import gc
import re
import hazm
import time
import json
import sacrebleu
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

import transformers
from transformers import AutoConfig, AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import MT5Config, MT5ForConditionalGeneration, MT5Tokenizer

from cleantext import clean

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

print()
print('numpy', np.__version__)
print('pandas', pd.__version__)
print('transformers', transformers.__version__)
print('torch', torch.__version__)
print()

# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")



numpy 1.19.5
pandas 1.1.5
transformers 4.7.0
torch 1.9.0+cu102

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [5]:
class MachineTranslationDataset(torch.utils.data.Dataset):
    """ Create a PyTorch dataset for Machine Translation. """

    def __init__(self, original_text, translated_text, tokenizer, max_length):
        self.original_text = original_text
        self.translated_text = translated_text
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.original_text)

    def __getitem__(self, item):
        encoding = self.tokenizer(
            self.original_text[item],
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors="pt"
        )
        inputs = {
            'original': self.original_text[item],
            'translated': self.translated_text[item],
            'input_ids': encoding.input_ids.flatten(),
            'attention_mask': encoding.attention_mask.flatten()
        }
        return inputs


class MachineTranslation:
    def __init__(self, model_name, model_type):
        self.normalizer = hazm.Normalizer()
        self.model_name = model_name
        if model_type.lower() == "mt5":
            self.tokenizer = MT5Tokenizer.from_pretrained(model_name)
            self.model = MT5ForConditionalGeneration.from_pretrained(model_name)
            self.config = MT5Config.from_pretrained(self.model_name)
        else:
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            self.model = AutoModelForSequenceClassification.from_pretrained(self.model_name)
            self.config = AutoConfig.from_pretrained(self.model_name)
            self.id2label = self.config.id2label
            self.label2id = self.config.label2id

    def load_dataset_test_file(self, dataset_name, dataset_file, **kwargs):
        if dataset_name.lower() == "mizan":
            if not os.path.exists(dataset_file):
                print(f'{dataset_file} not exists!')
                return
            data = pd.read_csv(dataset_file, delimiter="\t", names=['original', 'translation'], header=None)
            original, translation = data['original'].values.tolist(), data['translation'].values.tolist()
            print(f'test part:\n #original: {len(original)}, #translation: {len(translation)}')
            return original, translation
        if dataset_name.lower() == "combined":
            if not os.path.exists(dataset_file):
                print(f'{dataset_file} not exists!')
                return
            data = pd.read_csv(dataset_file, delimiter="\t", names=['original', 'translation', 'source'], header=None)
            if 'source' in kwargs:
                data = data[data['source'] == kwargs['source']]
                data = data[['original', 'translation']]
            else:
                data = data[['original', 'translation']]
            original, translation = data['original'].values.tolist(), data['translation'].values.tolist()
            print(f'test part:\n #original: {len(original)}, #translation: {len(translation)}')
            return original, translation

    def load_dataset_file(self, dataset_name, dataset_file, **kwargs):
        if dataset_name.lower() == "quran" or dataset_name.lower() == "bible":
            if not os.path.exists(dataset_file):
                print(f'{dataset_file} not exists!')
                return
            original, translation = [], []
            with open(dataset_file, encoding="utf8") as infile:
                for line in infile:
                    parts = line.strip().split('\t')
                    original.append(parts[0])
                    translation.append(parts[1])
            print(f'all data:\n #original: {len(original)}, #translation: {len(translation)}')

            _, original_test, _, translation_test = train_test_split(original, translation, test_size=0.1,
                                                                     random_state=1)
            print(f'test part:\n #original: {len(original_test)}, #translation: {len(translation_test)}')
            return original, translation, original_test, translation_test

    def mt5_machine_translation_inference(self, input_text, device):
        if not self.model or not self.tokenizer:
            print('Something wrong has been happened!')
            return

        tokenized_batch = self.tokenizer(
            input_text,
            padding=True,
            return_tensors="pt"
        )

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()

        input_ids = tokenized_batch.input_ids.to(device)
        attention_mask = tokenized_batch.attention_mask.to(device)
        outputs = self.model.generate(input_ids=input_ids,
                                      attention_mask=attention_mask)
        predictions = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)
        return predictions

    def mt5_evaluation(self, input_text, translated_text, device, max_length, split_reference=None, batch_size=4):
        if not self.model or not self.tokenizer:
            print('Something wrong has been happened!')
            return
        if len(input_text) != len(translated_text):
            print('length of inputs and its translations is not equal!!')
            return

        dataset = MachineTranslationDataset(original_text=input_text, translated_text=translated_text,
                                            tokenizer=self.tokenizer, max_length=max_length)
        data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
        print(f'#original_text:{len(input_text)}, #translated_text:{len(translated_text)}')
        print("#batch:", len(data_loader))

        if split_reference is None:
            max_num_ref = 1
        else:
            max_num_ref = 0
            for ref in translated_text:
                max_num_ref = max(max_num_ref, len(ref.split(split_reference)))
        print("#maximum_translation_reference:", max_num_ref)

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()

        total_time = 0
        output_predictions = []
        golden_translations, predicted_translations = [[] for _ in range(max_num_ref)], []
        print("Start to evaluate test data ...")
        for step, batch in enumerate(data_loader):
            b_input_ids = batch['input_ids']
            b_attention_mask = batch['attention_mask']

            # move tensors to GPU if CUDA is available
            b_input_ids = b_input_ids.to(device)
            b_attention_mask = b_attention_mask.to(device)

            # This will return the loss (rather than the model output) because we have provided the `labels`.
            with torch.no_grad():
                start = time.monotonic()
                b_outputs = self.model.generate(input_ids=b_input_ids, attention_mask=b_attention_mask)
                end = time.monotonic()
                total_time += end - start
                print(f'inference time for step {step}: {end - start}')

            b_predictions = self.tokenizer.batch_decode(b_outputs, skip_special_tokens=True)

            for i in range(len(b_input_ids)):
                if split_reference is None:
                    sample_golden_translation = [batch['translated'][i]]
                else:
                    sample_golden_translation = batch['translated'][i].split(split_reference)
                sample_generated_translation = b_predictions[i]
                bleu_score = sacrebleu.corpus_bleu(sys_stream=[sample_generated_translation],
                                                   ref_streams=[[g] for g in sample_golden_translation]).score
                output_predictions.append((batch['original'][i], batch['translated'][i], b_predictions[i], bleu_score))

                for j in range(max_num_ref):
                    try:
                        golden_translations[j].append(sample_golden_translation[j])
                    except:
                        golden_translations[j].append('')
                predicted_translations.append(sample_generated_translation)

        print("total inference time:", total_time)
        print("total inference time / #samples:", total_time / len(input_text))

        # evaluate
        print("BLEU Score: {}".format(sacrebleu.corpus_bleu(
            sys_stream=predicted_translations, ref_streams=golden_translations).score))
        return output_predictions


In [7]:
model_name='persiannlp/mt5-large-parsinlu-translation_en_fa'
mt_model = MachineTranslation(model_name=model_name, model_type="mt5")
print(mt_model.config)


MT5Config {
  "_name_or_path": "/home/patrick/hugging_face/t5/mt5-large",
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 2816,
  "d_kv": 64,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 24,
  "num_heads": 16,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "transformers_version": "4.7.0",
  "use_cache": true,
  "vocab_size": 250112
}



## Sample Inference

In [ ]:
input_list = [
  "Praise be to Allah, the Cherisher and Sustainer of the worlds;",
  "shrouds herself in white and walks penitentially disguised as brotherly love through factories and parliaments; offers help, but desires power;",
  "He thanked all fellow bloggers and organizations that showed support.",
  "Races are held between April and December at the Veliefendi Hippodrome near Bakerky, 15 km (9 miles) west of Istanbul.",
  "I want to pursue PhD in Computer Science about social network,what is the open problem in social networks?"
]
mt_model.mt5_machine_translation_inference(input_list, device)

['خدا را شکر که آفریننده و نگهدار جهان است.',
 'خود را با کفن سفید می پوشد و به شکل برادرانه ای در',
 'او از همه ی وبلاگ نویسان و سازمان هایی که از او حمایت کردند',
 'مسابقات بین آوریل و دسامبر در فرودگاه والی عبدین نزدیک بی',
 'من می خواهم پایان نامه دکتری را در رشته علوم کامپیوتر در']

## Mizan Dataset


In [ ]:
!wget https://media.githubusercontent.com/media/persiannlp/parsinlu/master/data/translation/mizan/mizan_test_en_fa.tsv
!ls

--2021-07-04 05:52:04--  https://media.githubusercontent.com/media/persiannlp/parsinlu/master/data/translation/mizan/mizan_test_en_fa.tsv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2296459 (2.2M) [text/plain]
Saving to: ‘mizan_test_en_fa.tsv’

mizan_test_en_fa.ts 100%[===================>]   2.19M  --.-KB/s    in 0.04s   

2021-07-04 05:52:05 (56.5 MB/s) - ‘mizan_test_en_fa.tsv’ saved [2296459/2296459]

adc.json  mizan_test_en_fa.tsv	sample_data


In [ ]:
mizan_original_text, mizan_translated_text = mt_model.load_dataset_test_file(dataset_name="mizan", dataset_file="mizan_test_en_fa.tsv")
print(mizan_original_text[:5])
print(mizan_translated_text[:5])
print(len(mizan_original_text))
print(len(mizan_translated_text))

test part:
 #original: 10000, #translation: 10000
['a sure sign with him of mental agitation.', ';Cesare, you are worried about this affair,; she said at last.', ';I am very sorry you feel so despondent over it;', 'but I could decide only as seemed right to me.;', ';It is not the affair,; he answered, sullenly; ;I know nothing about it,']
['این نشان قاطعی از هیجان وی بود شروع به قدم زدن در اتاق کرد.', 'عاقبت گفت: سزار! تو از این موضوع نگران شده\u200cای؛', 'بسیار متأسفم که نسبت به آن احساس بی اعتمادی می\u200cکنی؛', 'اما من فقط بدان علت نتوانستم تصمیم بگیرم که آن کار به نظرم صحیح رسید.', 'مارتینی، با اخم، پاسخ داد: به خاطر آن موضوع نیست، من در آن باره چیزی نمی\u200cدانم؛']
10000
10000


In [ ]:
mt_model.mt5_machine_translation_inference(mizan_original_text[:5], device)

['نشانه ای از آشفتگی ذهنی.',
 'سرانجام گفت: سزار تو نگران این موضوع هستی.',
 'متاسفم که این قدر احساس دلسردی می کنید.',
 'اما من فقط می توانم آنچه را که به نظرم درست می آمد،',
 'او با ترشرویی پاسخ داد: موضوع این نیست. من چیزی در این']

In [ ]:
!nvidia-smi
!lscpu

Sun Jul  4 05:52:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    35W /  70W |   6004MiB / 15109MiB |     14%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(mizan_original_text, mizan_translated_text, device, max_length=512, batch_size=64)

#original_text:10000, #translated_text:10000
#batch: 157
#maximum_translation_reference: 1
Start to evaluate test data ...
inference time for step 0: 11.002560764000009
inference time for step 1: 11.197749460000068
inference time for step 2: 11.381437555999923
inference time for step 3: 11.595092555000065
inference time for step 4: 11.847988691999944
inference time for step 5: 12.138475006000021
inference time for step 6: 12.407238092000057
inference time for step 7: 12.67074443499996
inference time for step 8: 12.532102395000038
inference time for step 9: 12.335915020000016
inference time for step 10: 12.220084821
inference time for step 11: 12.159990496999967
inference time for step 12: 12.19534670500002
inference time for step 13: 12.27234592900004
inference time for step 14: 12.313153003000025
inference time for step 15: 12.377726091999989
inference time for step 16: 12.368446500999994
inference time for step 17: 12.334870697000042
inference time for step 18: 12.357716787000072
inf

In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

a sure sign with him of mental agitation.	این نشان قاطعی از هیجان وی بود شروع به قدم زدن در اتاق کرد.	نشانه ای از آشفتگی ذهنی.	2.153749340017052
;Cesare, you are worried about this affair,; she said at last.	عاقبت گفت: سزار! تو از این موضوع نگران شده‌ای؛	سرانجام گفت: سزار تو نگران این موضوع هستی.	19.331263581394154
;I am very sorry you feel so despondent over it;	بسیار متأسفم که نسبت به آن احساس بی اعتمادی می‌کنی؛	متاسفم که این قدر احساس دلسردی می کنید.	5.0735520042259505
but I could decide only as seemed right to me.;	اما من فقط بدان علت نتوانستم تصمیم بگیرم که آن کار به نظرم صحیح رسید.	اما من فقط می توانم آنچه را که به نظرم درست می آمد،	12.014605158792346
;It is not the affair,; he answered, sullenly; ;I know nothing about it,	مارتینی، با اخم، پاسخ داد: به خاطر آن موضوع نیست، من در آن باره چیزی نمی‌دانم؛	او با ترشرویی پاسخ داد: موضوع این نیست. من چیزی در این	10.903051505700674
and it probably is all right, once you have consented to go into it.	هنگامی که تو قبول می‌کنی در کاری شرکت ج

In [ ]:
output_file_name = "translation_en-fa_mizan_test_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

## Quran Dataset

In [ ]:
!wget https://media.githubusercontent.com/media/persiannlp/parsinlu/master/data/translation/quran/quran_en_fa.tsv
!ls

--2021-07-04 06:24:24--  https://media.githubusercontent.com/media/persiannlp/parsinlu/master/data/translation/quran/quran_en_fa.tsv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16215125 (15M) [text/plain]
Saving to: ‘quran_en_fa.tsv’

quran_en_fa.tsv     100%[===================>]  15.46M  68.1MB/s    in 0.2s    

2021-07-04 06:24:25 (68.1 MB/s) - ‘quran_en_fa.tsv’ saved [16215125/16215125]

adc.json
mizan_test_en_fa.tsv
quran_en_fa.tsv
sample_data
translation_en-fa_mizan_test_persiannlp-mt5-large-parsinlu-translation_en_fa_outputs.txt


In [ ]:
quran_original_all, quran_translated_all, quran_original_test, quran_translated_test = mt_model.load_dataset_file(dataset_name="quran", dataset_file="quran_en_fa.tsv")
print(quran_original_all[0])
print(quran_translated_all[0])
print(quran_original_test[0])
print(quran_translated_test[0])

all data:
 #original: 6236, #translation: 6236
test part:
 #original: 624, #translation: 624
In the name of Allah, Most Gracious, Most Merciful.
به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خداى بخشاینده مهربان‌///به نام خداوند هستى بخش مهربان///به نام خداوند رحمتگر مهربان///به نام خداوند بخشنده‌ى مهربان.///به نام خداوند بخشنده مهربان///به نام خداوند بخشنده مهربان‌///به نام خداوند بخشنده بخشایشگر///بنام خداوند بخشاینده مهربان‌///به نام خدای رحمتگر بر آفریدگان، رحمتگر بر ویژگان‌
Nay, but man doth transgress all bounds,
این چنین نیست [که انسان سپاس گزار باشد] مسلما انسان سرکشی می‌کند.///این چنین نیست [که انسان سپاس گزار باشد] مسلما انسان سرکشی می‌کند.///حقا که آدمى نافرمانى مى‌کند،///چنین نیست [که مى‌پندارند]، بى‌شک آدمى طغیان مى‌کند///حقا که انسان سرکشى مى‌کند،///این چنین نیست [که انسان سپاس‌گزار باشد]، بى‌گمان آدمى سرکشى مى‌کند.///راستی که انسان سرکش و مغرور می‌شود.///چنین نیست، بی‌گمان انسان سر به طغیان برآورد

In [ ]:
mt_model.mt5_machine_translation_inference(quran_original_test[:5], device)

['نه، اما انسان از تمام محدودیت ها می گریزد.',
 'مصری ها می گویند: "اگر ثابت شود که دروغ گفته اید،',
 '"بنابراین هر دو به سوی او بروید و بگویید: "ما',
 'اما آن هایی که ایمان را رد می کنند و در حالی که با رد کردن آن',
 'دعا می کنم: "خدای من! استخوان های من واقعا ضع']

### test set

In [ ]:
!nvidia-smi
!lscpu

Sun Jul  4 06:24:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    43W /  70W |   6054MiB / 15109MiB |      4%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(quran_original_test, quran_translated_test, device, max_length=512, split_reference='///', batch_size=64)

#original_text:624, #translated_text:624
#batch: 10
#maximum_translation_reference: 11
Start to evaluate test data ...
inference time for step 0: 12.274264437000056
inference time for step 1: 12.465308603999802
inference time for step 2: 12.502336938000099
inference time for step 3: 12.424663454999973
inference time for step 4: 12.373488930999883
inference time for step 5: 12.327627927999856
inference time for step 6: 12.297784332999981
inference time for step 7: 12.309309653000128
inference time for step 8: 12.317210949000128
inference time for step 9: 9.368944033999924
total inference time: 120.66093926199983
total inference time / #samples: 0.19336688984294845
BLEU Score: 5.96828114663289


In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

Nay, but man doth transgress all bounds,	این چنین نیست [که انسان سپاس گزار باشد] مسلما انسان سرکشی می‌کند.///این چنین نیست [که انسان سپاس گزار باشد] مسلما انسان سرکشی می‌کند.///حقا که آدمى نافرمانى مى‌کند،///چنین نیست [که مى‌پندارند]، بى‌شک آدمى طغیان مى‌کند///حقا که انسان سرکشى مى‌کند،///این چنین نیست [که انسان سپاس‌گزار باشد]، بى‌گمان آدمى سرکشى مى‌کند.///راستی که انسان سرکش و مغرور می‌شود.///چنین نیست، بی‌گمان انسان سر به طغیان برآورد///چنین نیست (که شما می‌پندارید) به یقین انسان طغیان می‌کند،///نه چنین است همانا انسان سرکشی کند///چنان نیست (که انسان سرنهد). همواره انسان بی‌گمان سرکشی می‌کند،	نه، اما انسان از تمام محدودیت ها می گریزد.	4.515183688171633
(The Egyptians) said: "What then shall be the penalty of this, if ye are (proved) to have lied?"	گماشتگان گفتند: اگر شما دروغگو باشید [و سارق در میان شما یافت شود] کیفرش چیست؟///گماشتگان گفتند: اگر شما دروغگو باشید [و سارق در میان شما یافت شود] کیفرش چیست؟///گفتند: اگر دروغ گفته باشید جزاى دزد چیست؟///گفتند: پس اگر دروغ گفته باشید کیف

In [ ]:
output_file_name = "translation_en-fa_quran_test_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### all dataset

In [ ]:
!nvidia-smi
!lscpu

Sun Jul  4 06:26:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    34W /  70W |  13054MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(quran_original_all, quran_translated_all, device, max_length=512, split_reference='///', batch_size=64)

#original_text:6236, #translated_text:6236
#batch: 98
#maximum_translation_reference: 11
Start to evaluate test data ...
inference time for step 0: 12.332156836000195
inference time for step 1: 12.443184072999884
inference time for step 2: 12.473117930000171
inference time for step 3: 12.456999672000165
inference time for step 4: 12.415447734000281
inference time for step 5: 12.345577133000006
inference time for step 6: 12.332909514999756
inference time for step 7: 12.368086025000139
inference time for step 8: 12.364741988999867
inference time for step 9: 12.384374599999774
inference time for step 10: 12.43410830699986
inference time for step 11: 12.409041299999899
inference time for step 12: 12.38711603999991
inference time for step 13: 12.420894943000349
inference time for step 14: 12.442051582000204
inference time for step 15: 12.40110528699961
inference time for step 16: 12.390708284000084
inference time for step 17: 12.415991091999786
inference time for step 18: 12.41354661699961


In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

In the name of Allah, Most Gracious, Most Merciful.	به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خداى بخشاینده مهربان‌///به نام خداوند هستى بخش مهربان///به نام خداوند رحمتگر مهربان///به نام خداوند بخشنده‌ى مهربان.///به نام خداوند بخشنده مهربان///به نام خداوند بخشنده مهربان‌///به نام خداوند بخشنده بخشایشگر///بنام خداوند بخشاینده مهربان‌///به نام خدای رحمتگر بر آفریدگان، رحمتگر بر ویژگان‌	به نام خداوند بخشنده و مهربان.	50.81327481546149
Praise be to Allah, the Cherisher and Sustainer of the worlds;	همه ستایش‌ها، ویژه خدا، مالک و مربی جهانیان است.///همه ستایش‌ها، ویژه خدا، مالک و مربی جهانیان است.///ستایش خدا را که پروردگار جهانیان است.///ستایش خداى را که پروردگار جهانیان است///ستایش خدایى را که پروردگار جهانیان،///سپاس و ستایش مخصوص خداوندى است که پروردگار جهانیان است.///ستایش خدای را که پروردگار جهانیان است.///سپاس خداوند را که پروردگار جهانیان است‌///ستایش مخصوص خداوندی است که پروردگار جهانیان است.///سپاس خدایر

In [ ]:
output_file_name = "translation_en-fa_quran_all_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

## Bible Dataset

In [ ]:
!wget https://media.githubusercontent.com/media/persiannlp/parsinlu/master/data/translation/bible/bible_en_fa.tsv
!ls

--2021-07-06 04:42:00--  https://media.githubusercontent.com/media/persiannlp/parsinlu/master/data/translation/bible/bible_en_fa.tsv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10023337 (9.6M) [text/plain]
Saving to: ‘bible_en_fa.tsv’

bible_en_fa.tsv     100%[===================>]   9.56M  --.-KB/s    in 0.1s    

2021-07-06 04:42:01 (66.0 MB/s) - ‘bible_en_fa.tsv’ saved [10023337/10023337]

adc.json  bible_en_fa.tsv  sample_data


In [ ]:
bible_original_all, bible_translated_all, bible_original_test, bible_translated_test = mt_model.load_dataset_file(dataset_name="bible", dataset_file="bible_en_fa.tsv")
print(bible_original_all[0])
print(bible_translated_all[0])
print(bible_original_test[0])
print(bible_translated_test[0])

all data:
 #original: 31020, #translation: 31020
test part:
 #original: 3102, #translation: 3102
In the beginning God created the heaven and the earth.
در ابتدا، خدا آسمانها و زمین را آفرید.
And of the sons of Elizaphan; Shimri, and Jeiel: and of the sons of Asaph; Zechariah, and Mattaniah:
و چون قربانی های سوختنی تمام شد، پادشاه و جمیع حاضرین با وی رکوع کرده، سجده نمودند.


In [ ]:
mt_model.mt5_machine_translation_inference(bible_original_test[:5], device)

['و از پسران الیزایفان؛ شمری و جیل؛ و از پسران آ',
 'تمام ستون های اطراف دربار با سیم پوشیده می شوند. قلاب',
 'و موسی به آن ها گفت: اگر بچه های خرد و بچه های',
 'من به تو شیر داده ام نه گوشت: چون تا حالا قادر به',
 'و در پایان هفت روز، کلام خداوند به من رسید که می گوید']

### test set

In [ ]:
!nvidia-smi
!lscpu

Sun Jul  4 06:47:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    42W /  70W |   6034MiB / 15109MiB |     18%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(bible_original_test, bible_translated_test, device, max_length=512, batch_size=64)

#original_text:3102, #translated_text:3102
#batch: 49
#maximum_translation_reference: 1
Start to evaluate test data ...
inference time for step 0: 12.228137102000346
inference time for step 1: 12.658136698000362
inference time for step 2: 12.877489599
inference time for step 3: 12.646384457000295
inference time for step 4: 12.375103812000361
inference time for step 5: 12.244549648999964
inference time for step 6: 12.215947596000206
inference time for step 7: 12.262509632999809
inference time for step 8: 12.339390474000083
inference time for step 9: 12.437133364999681
inference time for step 10: 12.438785234000079
inference time for step 11: 12.384300747999987
inference time for step 12: 12.341263134000656
inference time for step 13: 12.323135816999638
inference time for step 14: 12.302042575999621
inference time for step 15: 12.286882862000311
inference time for step 16: 12.311633124999389
inference time for step 17: 12.328029919999608
inference time for step 18: 12.344602225999552
inf

In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

And of the sons of Elizaphan; Shimri, and Jeiel: and of the sons of Asaph; Zechariah, and Mattaniah:	و چون قربانی های سوختنی تمام شد، پادشاه و جمیع حاضرین با وی رکوع کرده، سجده نمودند.	و از پسران الیزایفان؛ شمری و جیل؛ و از پسران آ	2.35864181846352
All the pillars round about the court shall be filleted with silver; their hooks shall be of silver, and their sockets of brass.	«و تو بنی‌اسرائیل را امر فرما که روغن زیتون مصفی و کوبیده شده برای روشنایی نزد توبیاورند تا چراغها دائم روشن شود.در خیمه اجتماع، بیرون پرده‌ای که در برابر شهادت است، هارون و پسرانش از شام تا صبح، به حضورخداوند آن را درست کنند. و این برای بنی‌اسرائیل نسلا بعد نسل فریضه ابدی باشد.	تمام ستون های اطراف دربار با سیم پوشیده می شوند. قلاب	0.07327492496430335
And Moses said unto them, If the children of Gad and the children of Reuben will pass with you over Jordan, every man armed to battle, before the LORD, and the land shall be subdued before you; then ye shall give them the land of Gilead for a possession:	وبنی جاد، دیب

In [ ]:
output_file_name = "translation_en-fa_bible_test_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### all dataset

In [ ]:
!nvidia-smi
!lscpu

Tue Jul  6 04:42:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(bible_original_all, bible_translated_all, device, max_length=512, batch_size=64)

#original_text:31020, #translated_text:31020
#batch: 485
#maximum_translation_reference: 1
Start to evaluate test data ...
inference time for step 0: 11.15256517499995
inference time for step 1: 11.120832302000053
inference time for step 2: 11.274932879000062
inference time for step 3: 11.49848624599997
inference time for step 4: 11.733084300999963
inference time for step 5: 12.020071522999956
inference time for step 6: 12.230190402999938
inference time for step 7: 12.549069665999923
inference time for step 8: 12.880040401999963
inference time for step 9: 12.91952170500008
inference time for step 10: 12.645513418000064
inference time for step 11: 12.442045588000042
inference time for step 12: 12.382062661000077
inference time for step 13: 12.470690849999983
inference time for step 14: 12.631958149999946
inference time for step 15: 12.745012814999995
inference time for step 16: 12.671452211999963
inference time for step 17: 12.598149198999977
inference time for step 18: 12.5332633549999

In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

In the beginning God created the heaven and the earth.	در ابتدا، خدا آسمانها و زمین را آفرید.	در آغاز خداوند آسمان و زمین را خلق کرد.	17.747405280050266
And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters.	وزمین تهی و بایر بود و تاریکی بر روی لجه. و روح خدا سطح آبها را فرو گرفت.	و زمین بدون شکل و پوچ بود و تاریکی بر چهره ژرفا	12.045972307298896
And God said, Let there be light: and there was light.	و خدا گفت: «روشنایی بشود.» و روشنایی شد.	و خداوند گفت: بگذار روشن باشد: و روشن شد.	10.600313379512592
And God saw the light, that it was good: and God divided the light from the darkness.	و خدا روشنایی را دید که نیکوست و خداروشنایی را از تاریکی جدا ساخت.	و خداوند نور را دید که خوب است: و خداوند نور را از تاریکی	17.678748653651848
And God called the light Day, and the darkness he called Night. And the evening and the morning were the first day.	و خداروشنایی را روز نامید و تاریکی را شب نامید. وشام بود و ص

In [ ]:
output_file_name = "translation_en-fa_bible_all_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

## translation_combined

In [9]:
!mkdir translation_combined_en_fa
!wget https://media.githubusercontent.com/media/persiannlp/parsinlu/master/data/translation/translation_combined_en_fa/test.tsv -O translation_combined_en_fa/test.tsv
!ls translation_combined_en_fa

--2021-07-07 12:03:05--  https://media.githubusercontent.com/media/persiannlp/parsinlu/master/data/translation/translation_combined_en_fa/test.tsv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29348716 (28M) [text/plain]
Saving to: ‘translation_combined_en_fa/test.tsv’

translation_combine 100%[===================>]  27.99M  --.-KB/s    in 0.1s    

2021-07-07 12:03:06 (196 MB/s) - ‘translation_combined_en_fa/test.tsv’ saved [29348716/29348716]

test.tsv


### All combined test set

In [8]:
combined_original_test, combined_translated_test = mt_model.load_dataset_test_file(dataset_name="combined", dataset_file="translation_combined_en_fa/test.tsv")
print(combined_original_test[0])
print(combined_translated_test[0])

test part:
 #original: 48123, #translation: 48123
In the beginning God created the heaven and the earth.
در ابتدا، خدا آسمانها و زمین را آفرید.


In [ ]:
!nvidia-smi
!lscpu

Mon Jul  5 09:36:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(combined_original_test, combined_translated_test, device, max_length=512, batch_size=64)

#original_text:48123, #translated_text:48123
#batch: 752
#maximum_translation_reference: 1
Start to evaluate test data ...
inference time for step 0: 10.923964219000027
inference time for step 1: 10.940698411000085
inference time for step 2: 11.048636519999945
inference time for step 3: 11.230751957999928
inference time for step 4: 11.490831543000013
inference time for step 5: 11.77259734900008
inference time for step 6: 12.120391406999943
inference time for step 7: 12.452281303000063
inference time for step 8: 12.363276744999894
inference time for step 9: 12.126496762999977
inference time for step 10: 11.960564014000056
inference time for step 11: 11.966943148999917
inference time for step 12: 12.030916180000077
inference time for step 13: 12.152047855999854
inference time for step 14: 12.222405351999896
inference time for step 15: 12.203030542000079
inference time for step 16: 12.160304980000092
inference time for step 17: 12.121403107000106
inference time for step 18: 12.13745476899

In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

In the beginning God created the heaven and the earth.	در ابتدا، خدا آسمانها و زمین را آفرید.	در آغاز خداوند آسمان و زمین را خلق کرد.	17.747405280050266
And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters.	وزمین تهی و بایر بود و تاریکی بر روی لجه. و روح خدا سطح آبها را فرو گرفت.	و زمین بدون شکل و پوچ بود و تاریکی بر چهره ژرفا	12.045972307298896
And God said, Let there be light: and there was light.	و خدا گفت: «روشنایی بشود.» و روشنایی شد.	و خداوند گفت: بگذار روشن باشد: و روشن شد.	10.600313379512592
And God saw the light, that it was good: and God divided the light from the darkness.	و خدا روشنایی را دید که نیکوست و خداروشنایی را از تاریکی جدا ساخت.	و خداوند نور را دید که خوب است: و خداوند نور را از تاریکی	17.678748653651848
And God called the light Day, and the darkness he called Night. And the evening and the morning were the first day.	و خداروشنایی را روز نامید و تاریکی را شب نامید. وشام بود و ص

In [ ]:
output_file_name = "translation_en-fa_combined_all_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

In [ ]:
!nvidia-smi
!lscpu

Tue Jul  6 06:25:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    33W /  70W |  14430MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(combined_original_test, combined_translated_test, device, max_length=512, split_reference='///', batch_size=64)

#original_text:48123, #translated_text:48123
#batch: 752
#maximum_translation_reference: 11
Start to evaluate test data ...
inference time for step 0: 12.471139915000094
inference time for step 1: 13.006371201000547
inference time for step 2: 13.07603713699973
inference time for step 3: 12.719138563999877
inference time for step 4: 12.45461202200022
inference time for step 5: 12.346759501999259
inference time for step 6: 12.369885032000639
inference time for step 7: 12.503756516000067
inference time for step 8: 12.615309740999692
inference time for step 9: 12.68734966500051
inference time for step 10: 12.603571048999584
inference time for step 11: 12.552735064999979
inference time for step 12: 12.48833313599971
inference time for step 13: 12.489845354999488
inference time for step 14: 12.524385539999457
inference time for step 15: 12.548030228999778
inference time for step 16: 12.528940549000254
inference time for step 17: 12.566412743999535
inference time for step 18: 12.5640215640005

In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

In the beginning God created the heaven and the earth.	در ابتدا، خدا آسمانها و زمین را آفرید.	در آغاز خداوند آسمان و زمین را خلق کرد.	17.747405280050266
And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters.	وزمین تهی و بایر بود و تاریکی بر روی لجه. و روح خدا سطح آبها را فرو گرفت.	و زمین بدون شکل و پوچ بود و تاریکی بر چهره ژرفا	12.045972307298896
And God said, Let there be light: and there was light.	و خدا گفت: «روشنایی بشود.» و روشنایی شد.	و خداوند گفت: بگذار روشن باشد: و روشن شد.	10.600313379512592
And God saw the light, that it was good: and God divided the light from the darkness.	و خدا روشنایی را دید که نیکوست و خداروشنایی را از تاریکی جدا ساخت.	و خداوند نور را دید که خوب است: و خداوند نور را از تاریکی	17.678748653651848
And God called the light Day, and the darkness he called Night. And the evening and the morning were the first day.	و خداروشنایی را روز نامید و تاریکی را شب نامید. وشام بود و ص

In [ ]:
output_file_name = "translation_en-fa_combined_all_split-refs_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### Bible subset

In [10]:
bible_original_test, bible_translated_test = mt_model.load_dataset_test_file(dataset_name="combined", dataset_file="translation_combined_en_fa/test.tsv", source='bible_en_fa')
print(bible_original_test[0])
print(bible_translated_test[0])

test part:
 #original: 31020, #translation: 31020
In the beginning God created the heaven and the earth.
در ابتدا، خدا آسمانها و زمین را آفرید.


In [11]:
!nvidia-smi
!lscpu

Wed Jul  7 12:03:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    28W /  70W |   6012MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
evaluation_output = mt_model.mt5_evaluation(bible_original_test, bible_translated_test, device, max_length=512, batch_size=64)

#original_text:31020, #translated_text:31020
#batch: 485
#maximum_translation_reference: 1
Start to evaluate test data ...
inference time for step 0: 10.96086181399994
inference time for step 1: 10.993477852999945
inference time for step 2: 11.131020994999972
inference time for step 3: 11.174282500000004
inference time for step 4: 11.28506913000001
inference time for step 5: 11.39634447100002
inference time for step 6: 11.472904686999982
inference time for step 7: 11.616357778000065
inference time for step 8: 11.70694400800005
inference time for step 9: 11.801385557000003
inference time for step 10: 11.740368481000019
inference time for step 11: 11.827100707999989
inference time for step 12: 11.859941750000075
inference time for step 13: 11.872331802999952
inference time for step 14: 11.882651196999973
inference time for step 15: 11.943387873000006
inference time for step 16: 12.044160390999991
inference time for step 17: 12.131544598999994
inference time for step 18: 12.15386564799996

In [13]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

In the beginning God created the heaven and the earth.	در ابتدا، خدا آسمانها و زمین را آفرید.	در آغاز خداوند آسمان و زمین را خلق کرد.	17.747405280050266
And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters.	وزمین تهی و بایر بود و تاریکی بر روی لجه. و روح خدا سطح آبها را فرو گرفت.	و زمین بدون شکل و پوچ بود و تاریکی بر چهره ژرفا	12.045972307298896
And God said, Let there be light: and there was light.	و خدا گفت: «روشنایی بشود.» و روشنایی شد.	و خداوند گفت: بگذار روشن باشد: و روشن شد.	10.600313379512592
And God saw the light, that it was good: and God divided the light from the darkness.	و خدا روشنایی را دید که نیکوست و خداروشنایی را از تاریکی جدا ساخت.	و خداوند نور را دید که خوب است: و خداوند نور را از تاریکی	17.678748653651848
And God called the light Day, and the darkness he called Night. And the evening and the morning were the first day.	و خداروشنایی را روز نامید و تاریکی را شب نامید. وشام بود و ص

In [14]:
output_file_name = "translation_en-fa_combined_bible_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### Mizan subset

In [ ]:
mizan_original_test, mizan_translated_test = mt_model.load_dataset_test_file(dataset_name="combined", dataset_file="translation_combined_en_fa/test.tsv", source='mizan_test_en_fa')
print(mizan_original_test[0])
print(mizan_translated_test[0])

test part:
 #original: 10000, #translation: 10000
a sure sign with him of mental agitation.
این نشان قاطعی از هیجان وی بود شروع به قدم زدن در اتاق کرد.


In [ ]:
!nvidia-smi
!lscpu

Mon Jul  5 10:34:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P0    32W /  70W |  13182MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(mizan_original_test, mizan_translated_test, device, max_length=512, batch_size=64)

#original_text:10000, #translated_text:10000
#batch: 157
#maximum_translation_reference: 1
Start to evaluate test data ...
inference time for step 0: 12.054235607999999
inference time for step 1: 12.210684012000002
inference time for step 2: 12.016608636
inference time for step 3: 11.811473680999825
inference time for step 4: 11.704563029999917
inference time for step 5: 11.69403286800025
inference time for step 6: 11.740209104999849
inference time for step 7: 11.815247657000327
inference time for step 8: 11.860806224000044
inference time for step 9: 11.87651009000001
inference time for step 10: 11.839924285000052
inference time for step 11: 11.791757101000258
inference time for step 12: 11.754407600000377
inference time for step 13: 11.773208921000332
inference time for step 14: 11.788610859000073
inference time for step 15: 11.821155905000523
inference time for step 16: 11.831630820999635
inference time for step 17: 11.83101403099954
inference time for step 18: 11.829432986000029
inf

In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

a sure sign with him of mental agitation.	این نشان قاطعی از هیجان وی بود شروع به قدم زدن در اتاق کرد.	نشانه ای از آشفتگی ذهنی.	2.153749340017052
;Cesare, you are worried about this affair,; she said at last.	عاقبت گفت: سزار! تو از این موضوع نگران شده‌ای؛	سرانجام گفت: سزار تو نگران این موضوع هستی.	19.331263581394154
;I am very sorry you feel so despondent over it;	بسیار متأسفم که نسبت به آن احساس بی اعتمادی می‌کنی؛	متاسفم که این قدر احساس دلسردی می کنید.	5.0735520042259505
but I could decide only as seemed right to me.;	اما من فقط بدان علت نتوانستم تصمیم بگیرم که آن کار به نظرم صحیح رسید.	اما من فقط می توانم آنچه را که به نظرم درست می آمد،	12.014605158792346
;It is not the affair,; he answered, sullenly; ;I know nothing about it,	مارتینی، با اخم، پاسخ داد: به خاطر آن موضوع نیست، من در آن باره چیزی نمی‌دانم؛	او با ترشرویی پاسخ داد: موضوع این نیست. من چیزی در این	10.903051505700674
and it probably is all right, once you have consented to go into it.	هنگامی که تو قبول می‌کنی در کاری شرکت ج

In [ ]:
output_file_name = "translation_en-fa_combined_mizan_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### qqp subset

In [ ]:
qqp_original_test, qqp_translated_test = mt_model.load_dataset_test_file(dataset_name="combined", dataset_file="translation_combined_en_fa/test.tsv", source='qqp_train_en_fa')
print(qqp_original_test[0])
print(qqp_translated_test[0])

test part:
 #original: 1104, #translation: 1104
How can I slowly lose weight?
چگونه می توانم کم کم وزن خود را کاهش دهم؟


In [ ]:
!nvidia-smi
!lscpu

Mon Jul  5 10:29:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    31W /  70W |  13182MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(qqp_original_test, qqp_translated_test, device, max_length=512, batch_size=64)

#original_text:1104, #translated_text:1104
#batch: 18
#maximum_translation_reference: 1
Start to evaluate test data ...
inference time for step 0: 11.779408157000034
inference time for step 1: 12.016400186000283
inference time for step 2: 12.103958517999672
inference time for step 3: 11.918050682000285
inference time for step 4: 11.769366241999705
inference time for step 5: 11.692146346999834
inference time for step 6: 11.681737672000054
inference time for step 7: 11.721568475999902
inference time for step 8: 11.81902531600008
inference time for step 9: 11.844798481999987
inference time for step 10: 11.856273417000011
inference time for step 11: 11.82962239499966
inference time for step 12: 11.7953522050002
inference time for step 13: 11.77978865599971
inference time for step 14: 11.779463627999576
inference time for step 15: 11.793865744000414
inference time for step 16: 11.798024724000243
inference time for step 17: 3.1477236249997986
total inference time: 204.12657447199945
total in

In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

How can I slowly lose weight?	چگونه می توانم کم کم وزن خود را کاهش دهم؟	چگونه می توانم به آرامی وزنم را کم کنم؟	18.04438612975343
How does masturbation affect one's concentrating power?	چگونه استمناء بر قدرت تمرکز شخص تأثیر می گذارد؟	چگونه جلق زدن بر قدرت تمرکز اثر می گذارد؟	23.356898886410015
What does it mean when I dreamed of someone and said they're dying?	وقتی خواب کسی را دیدم و گفتم آنها در حال مرگ هستند ، چه معنایی دارد؟	معنی اینکه من خواب کسی را دیدم و گفتم آنها دارند می میر	37.34381622366602
How can I modify apk files?	چگونه می توانم فایل های apk را تغییر دهم؟	چگونه می توانم فايل های apk را تغییر دهم؟	59.694917920196445
Where can I buy first copy shoes in mumbai?	از کجا می توانم کفش های کپی درجه اول را در بمبئی بخرم؟	کجا می توانم کفش های کپی اول را در بمبئی بخرم؟	67.80814773941113
What is the step by step procedure to apply for Australian PR?	روش مرحله به مرحله برای درخواست شهروندی دایم استرالیا چیست؟	مرحله به مرحله تقاضای کار برای تبلیغات استرالیا چیست	18.04438612975343
What 

In [ ]:
output_file_name = "translation_en-fa_combined_qqp_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### Quran subset

In [ ]:
quran_original_test, quran_translated_test = mt_model.load_dataset_test_file(dataset_name="combined", dataset_file="translation_combined_en_fa/test.tsv", source='quran_en_fa')
print(quran_original_test[0])
print(quran_translated_test[0])

test part:
 #original: 5999, #translation: 5999
In the name of Allah, Most Gracious, Most Merciful.
به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خداى بخشاینده مهربان‌///به نام خداوند هستى بخش مهربان///به نام خداوند رحمتگر مهربان///به نام خداوند بخشنده‌ى مهربان.///به نام خداوند بخشنده مهربان///به نام خداوند بخشنده مهربان‌///به نام خداوند بخشنده بخشایشگر///بنام خداوند بخشاینده مهربان‌///به نام خدای رحمتگر بر آفریدگان، رحمتگر بر ویژگان‌


In [ ]:
!nvidia-smi
!lscpu

Mon Jul  5 09:46:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P0    32W /  70W |  13054MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_output = mt_model.mt5_evaluation(quran_original_test, quran_translated_test, device, max_length=512, batch_size=64)

#original_text:5999, #translated_text:5999
#batch: 94
#maximum_translation_reference: 1
Start to evaluate test data ...
inference time for step 0: 12.024303890999818
inference time for step 1: 12.266677918999903
inference time for step 2: 12.064602577999949
inference time for step 3: 11.908859637999967
inference time for step 4: 11.786815170999944
inference time for step 5: 11.782907660000092
inference time for step 6: 11.797222325999883
inference time for step 7: 11.877382120999982
inference time for step 8: 11.943228240000053
inference time for step 9: 11.939584618000026
inference time for step 10: 11.95920186300009
inference time for step 11: 11.89089985999999
inference time for step 12: 11.848802604000184
inference time for step 13: 11.841934273000106
inference time for step 14: 11.833445453000195
inference time for step 15: 11.887068074999888
inference time for step 16: 11.87818920399991
inference time for step 17: 11.892075033000083
inference time for step 18: 11.876865818999931


In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

In the name of Allah, Most Gracious, Most Merciful.	به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خداى بخشاینده مهربان‌///به نام خداوند هستى بخش مهربان///به نام خداوند رحمتگر مهربان///به نام خداوند بخشنده‌ى مهربان.///به نام خداوند بخشنده مهربان///به نام خداوند بخشنده مهربان‌///به نام خداوند بخشنده بخشایشگر///بنام خداوند بخشاینده مهربان‌///به نام خدای رحمتگر بر آفریدگان، رحمتگر بر ویژگان‌	به نام خداوند بخشنده و مهربان.	6.486055269835277e-05
Praise be to Allah, the Cherisher and Sustainer of the worlds;	همه ستایش‌ها، ویژه خدا، مالک و مربی جهانیان است.///همه ستایش‌ها، ویژه خدا، مالک و مربی جهانیان است.///ستایش خدا را که پروردگار جهانیان است.///ستایش خداى را که پروردگار جهانیان است///ستایش خدایى را که پروردگار جهانیان،///سپاس و ستایش مخصوص خداوندى است که پروردگار جهانیان است.///ستایش خدای را که پروردگار جهانیان است.///سپاس خداوند را که پروردگار جهانیان است‌///ستایش مخصوص خداوندی است که پروردگار جهانیان است.///سپاس خ

In [ ]:
output_file_name = "translation_en-fa_combined_quran_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

In [ ]:
evaluation_output = mt_model.mt5_evaluation(quran_original_test, quran_translated_test, device, max_length=512, split_reference='///', batch_size=64)

#original_text:5999, #translated_text:5999
#batch: 94
#maximum_translation_reference: 11
Start to evaluate test data ...
inference time for step 0: 12.273953555999924
inference time for step 1: 12.235470778000035
inference time for step 2: 12.006577334999747
inference time for step 3: 11.820588996999959
inference time for step 4: 11.768261607999648
inference time for step 5: 11.731273218000297
inference time for step 6: 11.82269385499967
inference time for step 7: 11.932606427000337
inference time for step 8: 11.980891961999987
inference time for step 9: 11.92699285000026
inference time for step 10: 11.92766253200034
inference time for step 11: 11.855454267000368
inference time for step 12: 11.853847365000092
inference time for step 13: 11.853064964999703
inference time for step 14: 11.874385013999927
inference time for step 15: 11.917686171000241
inference time for step 16: 11.890090248999968
inference time for step 17: 11.859109415999683
inference time for step 18: 11.87656835000007


In [ ]:
for text, true_translation, generated_translation, bleu_score in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}'.format(text, true_translation, generated_translation, bleu_score))

In the name of Allah, Most Gracious, Most Merciful.	به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خدا که رحمتش بی‌اندازه است و مهربانی‌اش همیشگی.///به نام خداى بخشاینده مهربان‌///به نام خداوند هستى بخش مهربان///به نام خداوند رحمتگر مهربان///به نام خداوند بخشنده‌ى مهربان.///به نام خداوند بخشنده مهربان///به نام خداوند بخشنده مهربان‌///به نام خداوند بخشنده بخشایشگر///بنام خداوند بخشاینده مهربان‌///به نام خدای رحمتگر بر آفریدگان، رحمتگر بر ویژگان‌	به نام خداوند بخشنده و مهربان.	50.81327481546149
Praise be to Allah, the Cherisher and Sustainer of the worlds;	همه ستایش‌ها، ویژه خدا، مالک و مربی جهانیان است.///همه ستایش‌ها، ویژه خدا، مالک و مربی جهانیان است.///ستایش خدا را که پروردگار جهانیان است.///ستایش خداى را که پروردگار جهانیان است///ستایش خدایى را که پروردگار جهانیان،///سپاس و ستایش مخصوص خداوندى است که پروردگار جهانیان است.///ستایش خدای را که پروردگار جهانیان است.///سپاس خداوند را که پروردگار جهانیان است‌///ستایش مخصوص خداوندی است که پروردگار جهانیان است.///سپاس خدایر

In [ ]:
output_file_name = "translation_en-fa_combined_quran_split-refs_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for text, true_translation, generated_translation, bleu_score in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\n'.format(text, true_translation, generated_translation, bleu_score))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

## QQP Dataset

## TEP Dataset


## OPUS Dataset

## Global Voice Dataset